In [43]:
import quandl
quandl.ApiConfig.api_key = "NG_msDKqFXScjydFHyzz"
data = quandl.get("CUR/HKD")

In [ ]:
import datetime
import requests
from time import sleep

def generate_forex_dataframe():
    '''Generates a dataframe with exchange rates for base currency USD'''

    # first possible date
    START_DATE = datetime.date(2000,1,1)

    # create forex dataframe
    allowed_currencies = ['GBP', 'EUR', 'AUD', 'JPY', 'HKD', 'INR', 'CNY', 'CHF', 'CAD', ]
    cols = ['date'] + allowed_currencies

    # set starting day
    d = START_DATE

    # create list for forex rows
    forex_rows = list()
    date_format = "%Y-%m-%d"

    while d != datetime.date.today():
        # get data
        url = 'https://api.fixer.io/' + d.strftime(date_format) + '?base=USD&symbols=EUR,GBP,AUD,CHF,CNY,HKD,JPY,CAD,INR'
        print(url)
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36', 'content-type': 'application/json'}
        #http_proxy  = "181.57.156.186:8080"
        #proxyDict = {"http"  : http_proxy}
        #r = requests.get(url, proxies= proxyDict, headers=headers)
        r = requests.get(url, headers=headers)
        json = r.json()     

        # put data in dict
        forex_row = dict()

        try:
            forex_row['date'] = date.strptime(json['date'], date_format)
        except:
            d += datetime.timedelta(days=1)
            break

        for cur in allowed_currencies:
            try:
                forex_row[cur] = json['rates'][cur]
            except: 
                forex_row[cur] = None

        # append dict to list
        forex_rows.append(forex_row)

        # set day to next day
        d += datetime.timedelta(days=1)
        sleep(0.5)

    # create forex dataframe
    forex = pd.DataFrame(forex_rows, columns = cols) 

    #forex.set_index('date')
    forex['date'] = pd.to_datetime(forex['date'], format = date_format)
    forex.set_index(pd.DatetimeIndex(forex['date']), inplace = True)

    # clean-up
    forex.drop_duplicates(inplace = True)
    forex.drop(['date'], axis=1, inplace=True)

    return forex

forex = generate_forex_dataframe()
#artpredict_path = "/Users/denizthemenace/dev/artcollect/"
#datapath = artpredict_path + "data/"
forex.to_csv(datapath + 'forex.csv')

In [ ]:
import requests
url = "http://api.fixer.io/2000-01-01?base=USD&symbols=EUR,GBP,AUD,CHF,CNY,HKD,JPY,CAD,INR"
#headers = "Mozilla/5.0"
#http_proxy  = "41.75.76.75:62225"
#https_proxy = "https://10.10.1.11:1080"
#ftp_proxy   = "ftp://10.10.1.10:3128"

proxyDict = { 
              "http"  : http_proxy, 
              #"https" : https_proxy, 
              #"ftp"   : ftp_proxy
            }

#r = requests.get(url, proxies=proxyDict)
r = requests.get(url)
json = r.json()  
print(json['rates']['CAD'])

In [1]:
# forex api version
def convert_to_usd(row):
    ''' Converts '''
    try:
        # if date doesn't exist, e.g. because of weekend, return closest value
        return row['price'] / forex.iloc[forex.index.get_loc(row['sale_date'] ,method='nearest')][row['currency']]
    except:
        return None
    

testdf['usdprice'] = testdf.apply(convert_to_usd, axis=1)




NameError: name 'testdf' is not defined